In [7]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [8]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs


In [9]:
#Add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

In [10]:
#Use the citipy module to determine city based on latitude and longitude
from citipy import citipy

In [11]:
#Create a list for holding the cities
cities = []

#Identify the nearest city for each latitude and longitude combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    #If the city is unique, then we will add it to the cities list
    if city not in cities:
        cities.append(city)
#Print the city count to confirm sufficient count
len(cities)

608

In [12]:
#Import dependencies
from citipy import citipy
import requests
from config import weather_api_key
from datetime import datetime

url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [13]:
#Create an empty list to hold the weather data
city_data = []

#Print the beginning of the logging
print("Beginning Data Retrieval     ")
print("-----------------------------")

#Create counters
record_count = 1
set_count = 1

#Loop through all the cities in our list
for i, city in enumerate(cities):
    
    #Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    #Create endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ","+")
    #Log the URL, record, and set numbers and the city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    #Add 1 toS the record count
    record_count += 1

#Run an API request for each of the cites 
    try:
        #Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()
        #Parse out the needed data
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_humidity = city_weather['main']['humidity']
        city_clouds = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        city_country = city_weather['sys']['country']
        #Convert the date to ISO standard
        city_date = datetime.utcfromtimestamp(city_weather['dt']).strftime('%Y-%m-%d %H:%M:%S')
        #Append the city information into city_data list
        city_data.append({'City': city.title(),
                          'Lat': city_lat, 
                          'Lng': city_lng, 
                          'Max Temp': city_max_temp, 
                          'Humidity': city_humidity, 
                          'Cloudiness': city_clouds, 
                          'Wind Speed': city_wind, 
                          'Country': city_country, 
                          'Date': city_date})
#If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass

#Indicate that the Data Loading is complete
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | hermanus
Processing Record 2 of Set 1 | mackay
Processing Record 3 of Set 1 | bandarbeyla
Processing Record 4 of Set 1 | busselton
Processing Record 5 of Set 1 | aneby
Processing Record 6 of Set 1 | port-gentil
Processing Record 7 of Set 1 | salalah
Processing Record 8 of Set 1 | atuona
Processing Record 9 of Set 1 | rikitea
Processing Record 10 of Set 1 | ushuaia
Processing Record 11 of Set 1 | waitati
Processing Record 12 of Set 1 | teknaf
Processing Record 13 of Set 1 | saint george
Processing Record 14 of Set 1 | cape town
Processing Record 15 of Set 1 | ribeira grande
Processing Record 16 of Set 1 | dikson
Processing Record 17 of Set 1 | barrow
Processing Record 18 of Set 1 | gaozhou
Processing Record 19 of Set 1 | georgetown
Processing Record 20 of Set 1 | butaritari
Processing Record 21 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 22 of Set 1 | bloemfontein
Processing

In [14]:
len(city_data)

561

In [15]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hermanus,-34.4187,19.2345,63.81,89,100,8.28,ZA,2022-04-14 00:01:32
1,Mackay,-21.1500,149.2000,78.62,64,96,15.59,AU,2022-04-14 00:00:14
2,Bandarbeyla,9.4942,50.8122,79.34,83,11,2.55,SO,2022-04-14 00:01:33
3,Busselton,-33.6500,115.3333,56.53,78,0,8.55,AU,2022-04-14 00:01:33
4,Aneby,57.8390,14.8102,44.60,90,100,5.73,SE,2022-04-14 00:01:34
5,Port-Gentil,-0.7193,8.7815,80.26,84,100,7.52,GA,2022-04-14 00:01:34
6,Salalah,17.0151,54.0924,77.97,73,0,3.65,OM,2022-04-14 00:00:00
7,Atuona,-9.8000,-139.0333,78.98,78,44,18.86,PF,2022-04-14 00:01:35
8,Rikitea,-23.1203,-134.9692,78.10,71,9,17.45,PF,2022-04-14 00:01:35
9,Ushuaia,-54.8000,-68.3000,47.01,93,74,5.61,AR,2022-04-14 00:01:36


In [ ]:
#Reorder the columns
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

In [ ]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"

# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")